In [ ]:
# script to compile calculations from new workflow

In [1]:
import os
import re
import glob

import rmgpy.chemkin
import rmgpy.data.kinetics
import rmgpy.data.thermo

import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict

# Add thermo

In [6]:
# also load new thermo:
entries = []
DFT_DIR = "/work/westgroup/harris.se/autoscience/reaction_calculator/dft/"
thermo_libs = glob.glob(os.path.join(DFT_DIR, 'thermo', 'species*', 'arkane', 'RMG_libraries'))
print(f'{len(thermo_libs)} thermo libs')

indices = [e.index for e in entries]
# Load the Arkane thermo
# entries = []
for i, lib_path in enumerate(thermo_libs):
    matches = re.search('species_([0-9]{4})', lib_path)
    species_index = int(matches[1])
    if species_index in indices:
#         print('skipping', i)
        continue
    else:
        print('adding', species_index)
    
    ark_thermo_database = rmgpy.data.thermo.ThermoDatabase()
    ark_thermo_database.load_libraries(
        lib_path,
    )

    for key in ark_thermo_database.libraries['thermo'].entries.keys():
        entry = ark_thermo_database.libraries['thermo'].entries[key]
        entry.index = species_index
        entry.label = entry.item.smiles
        entries.append(entry)
print(f'{len(entries)} entries')

130 thermo libs
adding 0
adding 4
adding 5
adding 6
adding 8
adding 9
adding 11
adding 12
adding 13
adding 14
adding 15
adding 16
adding 17
adding 18
adding 19
adding 20
adding 21
adding 22
adding 23
adding 24
adding 25
adding 26
adding 27
adding 28
adding 29
adding 30
adding 31
adding 32
adding 33
adding 34
adding 35
adding 36
adding 37
adding 38
adding 39
adding 41
adding 42
adding 43
adding 44
adding 45
adding 47
adding 48
adding 49
adding 50
adding 51
adding 52
adding 53
adding 55
adding 56
adding 57
adding 58
adding 59
adding 60
adding 61
adding 62
adding 63
adding 65
adding 66
adding 68
adding 70
adding 71
adding 72
adding 73
adding 74
adding 75
adding 76
adding 79
adding 81
adding 83
adding 84
adding 85
adding 88
adding 89
adding 90
adding 91
adding 92
adding 93
adding 94
adding 95
adding 96
adding 97
adding 98
adding 99
adding 100
adding 101
adding 102
adding 103
adding 104
adding 105
adding 106
adding 107
adding 108
adding 109
adding 110
adding 111
adding 112
adding 113
adding

In [10]:
# compile it all into a single database and a single library which I'll call harris_butane
ark_thermo_database = rmgpy.data.thermo.ThermoDatabase()
ark_thermo_database.libraries['thermo'] = rmgpy.data.thermo.ThermoLibrary()
ark_thermo_database.libraries['thermo'].label = 'harris_butane'
ark_thermo_database.libraries['thermo'].entries = OrderedDict()
for entry in entries:
    ark_thermo_database.libraries['thermo'].entries[entry.label] = entry

In [11]:
# save the results
ark_thermo_database.save_libraries('thermo')

# Add kinetics

In [18]:
# next, get kinetics from new workflow (overwrite old if repeates)
DFT_DIR = "/work/westgroup/harris.se/autoscience/reaction_calculator/dft/"
kinetics_libs = glob.glob(os.path.join(DFT_DIR, 'kinetics', 'reaction*', 'arkane', 'RMG_libraries'))
entries = []
# Load the Arkane kinetics
for i, lib_path in enumerate(kinetics_libs):

    matches = re.search('reaction_([0-9]{6})', lib_path)
    reaction_index = int(matches[1])
    ark_kinetics_database = rmgpy.data.kinetics.KineticsDatabase()
    ark_kinetics_database.load_libraries(lib_path)
    print(lib_path)
    # TODO fix bug related to load_libraries not getting the actual name
    existing_indices = [e.index for e in entries]
    for key in ark_kinetics_database.libraries[''].entries.keys():
        entry = ark_kinetics_database.libraries[''].entries[key]
        entry.index = reaction_index
        
        # overwrite existing
        try:
            place = existing_indices.index(reaction_index)
            entries[place] = entry
            print(f'Overwriting existing entry {reaction_index}')
        except ValueError:
            entries.append(entry)
            print(f'Adding new entry {reaction_index}')
            


/work/westgroup/harris.se/autoscience/reaction_calculator/dft/kinetics/reaction_000050/arkane/RMG_libraries
Adding new entry 50
/work/westgroup/harris.se/autoscience/reaction_calculator/dft/kinetics/reaction_000052/arkane/RMG_libraries
Adding new entry 52
/work/westgroup/harris.se/autoscience/reaction_calculator/dft/kinetics/reaction_000127/arkane/RMG_libraries
Adding new entry 127
/work/westgroup/harris.se/autoscience/reaction_calculator/dft/kinetics/reaction_000129/arkane/RMG_libraries
Adding new entry 129
/work/westgroup/harris.se/autoscience/reaction_calculator/dft/kinetics/reaction_000213/arkane/RMG_libraries
Adding new entry 213
/work/westgroup/harris.se/autoscience/reaction_calculator/dft/kinetics/reaction_000245/arkane/RMG_libraries
Adding new entry 245
/work/westgroup/harris.se/autoscience/reaction_calculator/dft/kinetics/reaction_000246/arkane/RMG_libraries
Adding new entry 246
/work/westgroup/harris.se/autoscience/reaction_calculator/dft/kinetics/reaction_000249/arkane/RMG_l

In [20]:
# compile it all into a single database and a single library which I'll call harris_butane
ark_kinetics_database = rmgpy.data.kinetics.KineticsDatabase()
ark_kinetics_database.libraries['kinetics'] = rmgpy.data.kinetics.KineticsLibrary()
ark_kinetics_database.libraries['kinetics'].label = 'harris_butane'
ark_kinetics_database.libraries['kinetics'].name = 'harris_butane'
ark_kinetics_database.libraries['kinetics'].entries = OrderedDict()
for entry in entries:
    ark_kinetics_database.libraries['kinetics'].entries[entry.label] = entry

In [21]:
# save the results
ark_kinetics_database.save_libraries('harris_kinetics', reindex=False)